# Library

In [1]:
import os
import cv2
import time
import numpy as np

from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort

import torch
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))

True
NVIDIA GeForce RTX 3050 Laptop GPU


# Data Prep

In [14]:
vids = [1,2,3]

for vid in vids:
    video_path = f'./source video/beyblade battle {vid} clean.mov'
    
    # Load the Video
    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)

    print(f"Processing video: beyblade battle {vid} clean.mov")
    print(f"FPS: {fps}")

    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    print(f"Total frames: {frame_count}")

    duration_seconds = frame_count / fps
    print(f"Duration: {duration_seconds} seconds")
    print()

Processing video: beyblade battle 1 clean.mov
FPS: 30.0
Total frames: 382
Duration: 12.733333333333333 seconds

Processing video: beyblade battle 2 clean.mov
FPS: 30.0
Total frames: 296
Duration: 9.866666666666667 seconds

Processing video: beyblade battle 3 clean.mov
FPS: 30.0
Total frames: 424
Duration: 14.133333333333333 seconds



In [15]:
output_dir = './output_frames'

for vid in vids:
    video_path = f'./source video/beyblade battle {vid} clean.mov'
    final_output_dir = os.path.join(output_dir, f'beyblade battle {vid} clean')
    if not os.path.exists(final_output_dir):
        os.makedirs(final_output_dir)

    frame_index = 0 
    cap = cv2.VideoCapture(video_path)

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break # end of video

        # save the frame as an image
        frame_filename = os.path.join(final_output_dir, f'frame_{frame_index:04d}.jpg')
        cv2.imwrite(frame_filename, frame)

        frame_index += 1

    cap.release()
    print(f'Done extracting frames from video beyblade battle {vid} clean.mov')

KeyboardInterrupt: 

From the frames extracted across the three videos, we will manually select 100 representative images that cover the full range of possible scenarios and variations.

In [ ]:
# === SETUP ===
image_folder = 'output_frames'  # Folder with extracted frames
model_path = 'yolo11n.pt'       # Replace with your trained model if needed
model = YOLO(model_path)

# Get list of frame files
frame_files = sorted([f for f in os.listdir(image_folder) if f.endswith('.jpg')])


100%|██████████| 5.35M/5.35M [00:02<00:00, 2.42MB/s]


In [ ]:
# === DETECT OBJECTS & TIME IT ===
total_time = 0
all_detections = {}

for idx, file_name in enumerate(frame_files):
    image_path = os.path.join(image_folder, file_name)
    frame = cv2.imread(image_path)

    start = time.time()
    results = model.predict(frame, imgsz=640, conf=0.5)[0]
    end = time.time()

    inference_time = end - start
    total_time += inference_time

    # Store bounding box info
    detections = []
    for box in results.boxes:
        x1, y1, x2, y2 = box.xyxy[0].tolist()
        conf = box.conf[0].item()
        cls = int(box.cls[0].item())
        detections.append({
            "bbox": [x1, y1, x2, y2],
            "confidence": conf,
            "class": cls
        })

    all_detections[file_name] = detections

    # Save prediction image with boxes in a unique filename
    img_with_boxes = results.plot()
    save_dir = './output_predictions'
    save_path = os.path.join(save_dir, file_name)
    cv2.imwrite(save_path, img_with_boxes)

# === SUMMARY ===
avg_time = total_time / len(frame_files)
print(f"\n✅ Processed {len(frame_files)} frames")
print(f"⏱️ Total time: {total_time:.2f} sec")
print(f"⚡ Avg inference per frame: {avg_time:.4f} sec")

0: 640x384 (no detections), 58.2ms
Speed: 2.2ms preprocess, 58.2ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 (no detections), 64.3ms
Speed: 2.7ms preprocess, 64.3ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 (no detections), 50.8ms
Speed: 2.7ms preprocess, 50.8ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 (no detections), 60.5ms
Speed: 2.3ms preprocess, 60.5ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 (no detections), 47.4ms
Speed: 2.4ms preprocess, 47.4ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 (no detections), 49.9ms
Speed: 2.3ms preprocess, 49.9ms inference, 0.8ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 (no detections), 81.1ms
Speed: 2.4ms preprocess, 81.1ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 (no detections), 50.2ms
Speed: 2.5ms preprocess, 50.2ms in

# Train Yolo V11

In [ ]:
# model = YOLO('yolo11s.pt')  # This will auto-download if it's a recognized name
model = YOLO('yolo11n.pt')  # This will auto-download if it's a recognized name

model.train(data="data.yaml", epochs=50, imgsz=920)

Ultralytics 8.3.146  Python-3.13.2 torch-2.7.0+cu126 CUDA:0 (NVIDIA GeForce RTX 3050 Laptop GPU, 4096MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=920, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train7, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=12.0, pretr

train: Scanning G:\My Drive\Lamaran Documents\AssistX Enterprise\Beyblade-Battle-Video-Analysis-System\train\labels... 236 images, 0 backgrounds, 0 corrupt: 100%|██████████| 236/236 [00:03<00:00, 67.43it/s]


train: New cache created: G:\My Drive\Lamaran Documents\AssistX Enterprise\Beyblade-Battle-Video-Analysis-System\train\labels.cache
val: Fast image access  (ping: 2.31.1 ms, read: 2.20.7 MB/s, size: 49.6 KB)


val: Scanning G:\My Drive\Lamaran Documents\AssistX Enterprise\Beyblade-Battle-Video-Analysis-System\valid\labels... 64 images, 0 backgrounds, 0 corrupt: 100%|██████████| 64/64 [00:02<00:00, 28.55it/s]


val: New cache created: G:\My Drive\Lamaran Documents\AssistX Enterprise\Beyblade-Battle-Video-Analysis-System\valid\labels.cache
Plotting labels to runs\detect\train7\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001667, momentum=0.9) with parameter groups 81 weight(decay=0.0), 88 weight(decay=0.0005), 87 bias(decay=0.0)
Image sizes 928 train, 928 val
Using 8 dataloader workers
Logging results to runs\detect\train7
Starting training for 50 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                   all         64        110      0.957      0.928      0.951      0.754



50 epochs completed in 0.625 hours.
Optimizer stripped from runs\detect\train7\weights\last.pt, 5.3MB
Optimizer stripped from runs\detect\train7\weights\best.pt, 5.3MB

Validating runs\detect\train7\weights\best.pt...
Ultralytics 8.3.146  Python-3.13.2 torch-2.7.0+cu126 CUDA:0 (NVIDIA GeForce RTX 3050 Laptop GPU, 4096MiB)
YOLO11n summary (fused): 100 layers, 2,582,542 parameters, 0 gradients, 6.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:01<00:00,  1.51it/s]


                   all         64        110      0.962      0.927      0.956      0.774
              beyblade         64         94      0.998      0.979      0.994      0.787
       broken beyblade         14         16      0.926      0.875      0.918      0.762
Speed: 1.7ms preprocess, 9.3ms inference, 0.0ms loss, 2.6ms postprocess per image
Results saved to runs\detect\train7


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x00000180C11F4590>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.0

# Load the Best Model

In [ ]:
vids = [1,2,3]
output_dir = './output_videos'
model = YOLO('./runs/detect/train7/weights/best.pt')  # Load the best model from training

In [ ]:
results = model.predict('./output_frames/beyblade battle 1 clean/frame_0100.jpg', conf=0.1)
results[0].show()


image 1/1 g:\My Drive\Lamaran Documents\AssistX Enterprise\Beyblade-Battle-Video-Analysis-System\output_frames\beyblade battle 1 clean\frame_0100.jpg: 384x640 1 beyblade, 43.1ms
Speed: 4.6ms preprocess, 43.1ms inference, 7.0ms postprocess per image at shape (1, 3, 384, 640)


In [ ]:
for vid in vids:
    # Prepare paths for each video
    video_path = f'./source video/beyblade battle {vid} clean.mov'
    final_output_dir = f'beyblade battle {vid} clean predictions.mp4'
    
    # Open the video using OpenCV
    cap = cv2.VideoCapture(video_path)

    # Get video properties
    fps = cap.get(cv2.CAP_PROP_FPS)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    # Define the codec and create VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec for mp4
    video_writer = cv2.VideoWriter(final_output_dir, fourcc, fps, (width, height))
    all_detections = {}

    # Iterate through the video frames
    frame_index = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break  # End of video

        # Perform object detection
        results = model.predict(frame, imgsz=640, conf=0.5)[0]

        # Draw bounding boxes and labels on the frame
        annotated_frame = results.plot()

        # Write the annotated frame to the output video
        video_writer.write(annotated_frame)

        frame_index += 1
        print(f"Processed frame {frame_index}/{total_frames}")

    # Release resources
    cap.release()
    video_writer.release()
    cv2.destroyAllWindows()
    print(f"Output video saved at {final_output_dir}")


0: 384x640 (no detections), 116.4ms
Speed: 5.2ms preprocess, 116.4ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)
Processed frame 1/382

0: 384x640 (no detections), 17.8ms
Speed: 3.6ms preprocess, 17.8ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)
Processed frame 2/382

0: 384x640 (no detections), 16.1ms
Speed: 2.7ms preprocess, 16.1ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)
Processed frame 3/382

0: 384x640 (no detections), 17.7ms
Speed: 3.0ms preprocess, 17.7ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)
Processed frame 4/382

0: 384x640 (no detections), 17.1ms
Speed: 2.9ms preprocess, 17.1ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)
Processed frame 5/382

0: 384x640 (no detections), 19.8ms
Speed: 3.2ms preprocess, 19.8ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)
Processed frame 6/382

0: 384x640 (no detections), 21.6ms
Speed: 3.3ms preprocess, 21.6ms 

# Object Tracking

In [49]:
from ultralytics import YOLO

In [ ]:
print(ultralytics.__file__)

c:\Users\Frags\anaconda3\envs\beyblade_analysis\Lib\site-packages\ultralytics\__init__.py


In [2]:
vids = [1,2,3]
output_dir = './output_videos'
model = YOLO('./runs/detect/train7/weights/best.pt')  # Load the best model from training

In [ ]:
results = model.track(
    source='./source video/beyblade battle 1 clean.mov',
    tracker='botsort.yaml',
    conf=0.5,
    save=True,
    verbose=False
)

## BOT SORT

In [ ]:
# Color histogram function
def get_color_histogram(image, box):
    x1, y1, x2, y2 = map(int, box)
    roi = image[y1:y2, x1:x2]
    hsv = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)
    hist = cv2.calcHist([hsv], [0, 1], None, [16, 16], [0, 180, 0, 256])
    cv2.normalize(hist, hist)
    return hist.flatten()

### Assign Unique IDs using Euclidian Distance

In [3]:
# Mapping between tracker ID and stable ID
stable_id_map = {}  # tracker_id -> stable_id
last_positions = {}  # stable_id -> (x_center, y_center)
next_stable_id = 1
max_stable_ids = 2

cap = cv2.VideoCapture('./source video/beyblade battle 1 clean.mov')

# Get video properties
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

# Define codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # or use 'XVID' for .avi
out = cv2.VideoWriter('./beyblade battle 1 clean tracking.mov', fourcc, fps, (width, height))

while True:
    ret, frame = cap.read()
    if not ret:
        break

    results = model.track(
        frame, 
        conf=0.5, 
        tracker='botsort.yaml',
        persist=True,
        verbose=False)

    boxes = results[0].boxes  # YOLO detections with BoT-SORT IDs

    for box in boxes:
        if box.id is None:
            continue

        track_id = int(box.id[0])
        x1, y1, x2, y2 = map(int, box.xyxy[0])
        x_center = (x1 + x2) / 2
        y_center = (y1 + y2) / 2

        # Assign stable ID
        if track_id not in stable_id_map:
            if next_stable_id <= max_stable_ids:
                stable_id_map[track_id] = next_stable_id
                next_stable_id += 1
            else:
                # Assign to closest stable ID based on distance
                dists = {
                    sid: np.linalg.norm(
                        np.array([x_center, y_center]) - np.array(last_positions.get(sid, [9999, 9999]))
                    ) for sid in range(1, max_stable_ids + 1)
                }
                stable_id_map[track_id] = min(dists, key=dists.get)

        stable_id = stable_id_map[track_id]
        last_positions[stable_id] = (x_center, y_center)

        # Draw bbox and stable ID
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(frame, f"Beyblade {stable_id}", (x1, y1 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 255, 0), 3)

    out.write(frame)

cap.release()
out.release()

### Assign IDs using Distance & Appearance

In [ ]:
# Mapping between tracker ID and stable ID
stable_id_map = {}  # tracker_id -> stable_id
last_positions = {}  # stable_id -> (x_center, y_center)
last_appearance = {}  # stable_id -> histogram
next_stable_id = 1
max_stable_ids = 2

cap = cv2.VideoCapture('./source video/beyblade battle 2 clean.mov')

# Get video properties
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

# Define codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # or use 'XVID' for .avi
out = cv2.VideoWriter('./beyblade battle 2 clean tracking.mov', fourcc, fps, (width, height))

while True:
    ret, frame = cap.read()
    if not ret:
        break

    results = model.track(
        frame, 
        conf=0.5, 
        tracker='botsort.yaml',
        persist=True,
        verbose=False)

    boxes = results[0].boxes  # YOLO detections with BoT-SORT IDs

    for box in boxes:
        if box.id is None:
            continue

        track_id = int(box.id[0])
        x1, y1, x2, y2 = map(int, box.xyxy[0])
        x_center = (x1 + x2) / 2
        y_center = (y1 + y2) / 2
        hist = get_color_histogram(frame, (x1,y1,x2,y2))

        # Assign stable ID
        if track_id not in stable_id_map:
            if next_stable_id <= max_stable_ids:
                stable_id = next_stable_id
                stable_id_map[track_id] = stable_id
                last_positions[stable_id] = (x_center, y_center)
                last_appearance[stable_id] = hist
                next_stable_id += 1
            else:
                # Match to best Stable ID based on motion + appearance
                best_sid = None
                best_score = float('inf')
                for sid in range(1, max_stable_ids + 1):
                    # Motion (Euclidean)
                    pos_score = np.linalg.norm(np.array([x_center, y_center]) - np.array(last_positions.get(sid, [9999, 9999])))
                    # Appearance (Bhattacharyya distance)
                    app_score = cv2.compareHist(hist.astype('float32'), last_appearance[sid].astype('float32'), cv2.HISTCMP_BHATTACHARYYA)
                    total_score = 0.5 * pos_score + 0.5 * app_score * 100  # weighted cost
                    if total_score < best_score:
                        best_score = total_score
                        best_sid = sid
                    stable_id_map[track_id] = best_sid            

        stable_id = stable_id_map[track_id]
        last_positions[stable_id] = (x_center, y_center)
        last_appearance[stable_id] = hist

        # Draw bbox and stable ID
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(frame, f"Beyblade {stable_id}", (x1, y1 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 255, 0), 3)

    out.write(frame)

cap.release()
out.release()

### Assigns IDs with Distance Improved

In [23]:
# Mapping between tracker ID and stable ID
stable_id_map = {}  # tracker_id -> stable_id
last_positions = {}  # stable_id -> (x_center, y_center)
next_stable_id = 1
max_stable_ids = 2

cap = cv2.VideoCapture('./source video/beyblade battle 2 clean.mov')

# Get video properties
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

# Define codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # or use 'XVID' for .avi
out = cv2.VideoWriter('./beyblade battle 2 clean tracking.mov', fourcc, fps, (width, height))

while True:
    ret, frame = cap.read()
    if not ret:
        break

    results = model.track(
        frame, 
        conf=0.5, 
        tracker='botsort.yaml',
        persist=True,
        verbose=False)

    boxes = results[0].boxes  # YOLO detections with BoT-SORT IDs

    for box in boxes:
        if box.id is None:
            continue

        track_id = int(box.id[0])
        x1, y1, x2, y2 = map(int, box.xyxy[0])
        x_center = (x1 + x2) / 2
        y_center = (y1 + y2) / 2

        # Assign stable ID
        if track_id not in stable_id_map:
            best_sid = None
            best_dist = float('inf')

            # Match new track_id to closest existing stable ID (only if not already taken)
            for sid in range(1, next_stable_id):
                if sid in last_positions:
                    dist = np.linalg.norm(
                        np.array([x_center, y_center]) - np.array(last_positions[sid])
                    )
                    if dist < best_dist and sid not in stable_id_map.values():
                        best_sid = sid
                        best_dist = dist

            if best_sid is not None and best_dist < 100:  # only accept if reasonable distance
                stable_id_map[track_id] = best_sid
            elif next_stable_id <= max_stable_ids:
                stable_id_map[track_id] = next_stable_id
                next_stable_id += 1
            else:
                # Too far and no new ID available: keep last mapping or skip
                continue


        stable_id = stable_id_map[track_id]
        last_positions[stable_id] = (x_center, y_center)

        # Draw bbox and stable ID
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(frame, f"Beyblade {stable_id}", (x1, y1 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 255, 0), 3)

    out.write(frame)

cap.release()
out.release()

### Assigns IDs without Distance

In [77]:
# Mapping between tracker ID and stable ID
stable_id_map = {}  # tracker_id -> stable_id
last_positions = {}  # stable_id -> (x_center, y_center)
next_stable_id = 1
max_stable_ids = 2

cap = cv2.VideoCapture('./source video/beyblade battle 1 clean.mov')

# Get video properties
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

# Define codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # or use 'XVID' for .avi
out = cv2.VideoWriter('./beyblade battle 1 clean tracking.mov', fourcc, fps, (width, height))

while True:
    ret, frame = cap.read()
    if not ret:
        break

    results = model.track(
        frame, 
        conf=0.5, 
        tracker='botsort.yaml',
        persist=True,
        verbose=False)

    boxes = results[0].boxes  # YOLO detections with BoT-SORT IDs

    for box in boxes:
        if box.id is None:
            continue

        track_id = int(box.id[0])
        x1, y1, x2, y2 = map(int, box.xyxy[0])
        x_center = (x1 + x2) / 2
        y_center = (y1 + y2) / 2

        # Assign stable ID
        if track_id not in stable_id_map:
            if next_stable_id <= max_stable_ids:
                stable_id_map[track_id] = next_stable_id
                next_stable_id += 1
            else:
                # Don't allow switching once 2 stable IDs are set
                continue

        stable_id = stable_id_map[track_id]
        last_positions[stable_id] = (x_center, y_center)

        # Draw bbox and stable ID
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(frame, f"Beyblade {stable_id}", (x1, y1 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 255, 0), 3)

    out.write(frame)

cap.release()
out.release()

## BOT SORT 2 Classes

In [53]:
# Mapping between tracker ID and stable ID
stable_id_map = {}  # tracker_id -> stable_id
last_positions = {}  # stable_id -> (x_center, y_center)
next_stable_id = 1
max_stable_ids = 2

cap = cv2.VideoCapture('./source video/beyblade battle 1 clean.mov')

# Get video properties
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

# Define codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # or use 'XVID' for .avi
out = cv2.VideoWriter('./beyblade battle 1 clean tracking.mov', fourcc, fps, (width, height))

# Suppose your two classes of interest are class IDs 0 and 1
target_classes = [0, 1]
class_names = {0: 'Beyblades', 1: 'Broken Beyblade'} 

while True:
    ret, frame = cap.read()
    if not ret:
        break

    results = model.track(
        frame, 
        conf=0.5, 
        tracker='botsort.yaml',
        persist=True,
        verbose=False)

    boxes = results[0].boxes  # YOLO detections with BoT-SORT IDs

    for box in boxes:
        if box.id is None:
            continue

        class_id = int(box.cls[0])  # get class ID, adapt if your model uses different attribute
        if class_id not in target_classes:
            continue  # skip other classes
        
        track_id = int(box.id[0])
        x1, y1, x2, y2 = map(int, box.xyxy[0])
        x_center = (x1 + x2) / 2
        y_center = (y1 + y2) / 2

        # Assign stable ID
        if track_id not in stable_id_map:
            if next_stable_id <= max_stable_ids:
                stable_id_map[track_id] = next_stable_id
                next_stable_id += 1
            else:
                # Assign to closest stable ID based on distance
                dists = {
                    sid: np.linalg.norm(
                        np.array([x_center, y_center]) - np.array(last_positions.get(sid, [9999, 9999]))
                    ) for sid in range(1, max_stable_ids + 1)
                }
                stable_id_map[track_id] = min(dists, key=dists.get)

        stable_id = stable_id_map[track_id]
        last_positions[stable_id] = (x_center, y_center)

        # Draw bbox and stable ID + class name
        color = (100, 100, 255) if class_id == 0 else (255, 150, 100)  # Red or Blue for example
        cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
        cv2.putText(frame, f"{class_names[class_id]} {stable_id}", (x1, y1 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 1.0, color, 2)

    out.write(frame)

cap.release()
out.release()